In [1]:
import pandas as pd
import typing
import pickle
import pandas as pd
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.exceptions import NotFittedError

import typing
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
import datetime
from scipy.spatial import KDTree

from baseline.raif_hack.metrics import metrics_stat, deviation_metric_one_sample
from service_func.floor import clean_floor
from service_func.utils import region_replacer, get_exchange_rate, get_investments_data_for_region
from service_func.federal_district import FEDERAL_DISTRICT

pd.set_option('display.max_columns', None)


In [2]:
ls

README.md       data/           service_data/
baseline/       raif_new.ipynb  service_func/


In [3]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
prices_df = pd.read_csv("data/prices.csv")

/Users/ulyumdzhi/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
train_df_full = pd.concat([train_df, test_df])

In [5]:
train_df_full = pd.merge(train_df_full, prices_df, on = 'id')

In [6]:
# Чистим "Этажность"
train_df_full['floor'] = train_df_full['floor'].fillna(0).astype('str')

text = train_df_full['floor']
text = text.apply(clean_floor)
train_df_full['floor'] = text
train_df_full['floor'] = train_df_full['floor'].astype('float').astype('int')
train_df_full['floor'] = [1 if row > 78 else row for row in train_df_full['floor']]

In [7]:
# Данные из Федеральной службы государственной статистики за 2019 год
# https://gks.ru/free_doc/new_site/population/bednost/tabl/1-2-6_2019.doc
# Локальное расположение файла 
s_path = 'service_data/salary.csv'
salary = pd.read_csv(s_path, sep=';')
salary['region'] = salary['region'].transform(region_replacer)

train_df_full = train_df_full.merge(salary, on='region', how='left')

In [8]:
train_df_full['federal_district'] = [str([key for key, val in FEDERAL_DISTRICT.items() if row in val]) if row == row else None for row in train_df_full['region']]
train_df_full['federal_district'] = [i[1] for i in train_df_full['federal_district']]
train_df_full['federal_district'] = train_df_full['federal_district'].astype(int)

In [9]:
# Собираем данные о курсах валют на даты, представленные в датасете
cur = ['EUR', 'USD']
exchange = get_exchange_rate(train_df_full, cur)

train_df_full = train_df_full.merge(exchange, on='date', how='left')


In [10]:
# Формируем таблицу по инвестиционной привлекательности регионов и добавляем эти данные
investments = get_investments_data_for_region(train_df_full)
train_df_full = train_df_full.merge(investments, on='region', how='left')


In [11]:
train_df_full.columns.to_list()

['city',
 'floor',
 'id',
 'lat',
 'lng',
 'osm_amenity_points_in_0.001',
 'osm_amenity_points_in_0.005',
 'osm_amenity_points_in_0.0075',
 'osm_amenity_points_in_0.01',
 'osm_building_points_in_0.001',
 'osm_building_points_in_0.005',
 'osm_building_points_in_0.0075',
 'osm_building_points_in_0.01',
 'osm_catering_points_in_0.001',
 'osm_catering_points_in_0.005',
 'osm_catering_points_in_0.0075',
 'osm_catering_points_in_0.01',
 'osm_city_closest_dist',
 'osm_city_nearest_name',
 'osm_city_nearest_population',
 'osm_crossing_closest_dist',
 'osm_crossing_points_in_0.001',
 'osm_crossing_points_in_0.005',
 'osm_crossing_points_in_0.0075',
 'osm_crossing_points_in_0.01',
 'osm_culture_points_in_0.001',
 'osm_culture_points_in_0.005',
 'osm_culture_points_in_0.0075',
 'osm_culture_points_in_0.01',
 'osm_finance_points_in_0.001',
 'osm_finance_points_in_0.005',
 'osm_finance_points_in_0.0075',
 'osm_finance_points_in_0.01',
 'osm_healthcare_points_in_0.005',
 'osm_healthcare_points_in_0.

In [12]:
TARGET = 'per_square_meter_price'
# признаки (или набор признаков), для которых применяем smoothed target encoding
CATEGORICAL_STE_FEATURES = ['region', 'street', 'city', 'realty_type']
# признаки, для которых применяем one hot encoding
CATEGORICAL_OHE_FEATURES = []
# численные признаки
NUM_FEATURES = ['lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
       'osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
       'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500','total_square']

NEW_NUM_FEATURES = [
       'median_salary', 'surv_minimum',
       'EUR', 'USD', 'potential_percent_2019', 'weighted_risk_2019']

NEW_CAT_FEATURES = ['federal_district', 'risk_rank_2019', 'potential_rank_2019']

MODEL_PARAMS = dict(
            n_estimators=5000,
            learning_rate=0.01,
            reg_alpha=1,
            num_leaves=40,
            min_child_samples=5,
            importance_type="gain",
            n_jobs=-1,
            random_state=563,
        )

In [13]:
def aggregate(df, column):
    temp_df = pd.DataFrame(df.groupby([column]).agg({'per_square_meter_price':[np.mean, np.median]})).reset_index() 
    temp_df.columns = [column, f'per_square_meter_price_{column}_mean', f'per_square_meter_price_{column}_median']
    return pd.merge(df, temp_df, on=column, how='left')

In [14]:
train_df_full = aggregate(train_df_full, 'city')
train_df_full = aggregate(train_df_full, 'region')
train_df_full = aggregate(train_df_full, 'street')
train_df_full = aggregate(train_df_full, 'realty_type')

In [15]:
#train_df = train_df[train_df['city'] == 'Москва']

In [16]:
le_region = LabelEncoder()
le_city = LabelEncoder()
le_street = LabelEncoder()
le_region = LabelEncoder()

In [17]:
train_df_full['region'] = le_region.fit_transform(train_df_full['region'])
train_df_full['city'] = le_city.fit_transform(train_df_full['city'])
train_df_full['street'] = train_df_full['street'].fillna('')
train_df_full['street'] =le_street.fit_transform(train_df_full['street'])

In [18]:
train_df_full['month'] = train_df_full['date'].apply(lambda x:datetime.datetime.strptime(x, "%Y-%m-%d").month)

In [19]:
FE = list(train_df_full.columns[-9:]) 

In [20]:
ss = StandardScaler()

In [21]:
train_df_full[NUM_FEATURES+NEW_NUM_FEATURES] = ss.fit_transform(train_df_full[NUM_FEATURES+NEW_NUM_FEATURES])

In [22]:
train_df_proc = train_df_full[train_df_full['id'].isin(train_df['id'])]
test_df_proc  = train_df_full[train_df_full['id'].isin(test_df['id'])]

In [23]:
RADUIS_FEATURES = list(prices_df.columns[1:])

In [24]:
train_split = train_df_proc[train_df_proc['date'] < '2020-07-01']
val_split = train_df_proc[train_df_proc['date'] >= '2020-07-01']

In [25]:
train_split_offer = train_split[train_split['price_type'] == 0]
train_split_manual = train_split[train_split['price_type'] == 1]
val_split_offer = val_split[val_split['price_type'] == 0]
val_split_manual = val_split[val_split['price_type'] == 1]

In [26]:
X_train_offer, y_train_offer =  train_split_offer[NUM_FEATURES+CATEGORICAL_OHE_FEATURES+CATEGORICAL_STE_FEATURES+FE+RADUIS_FEATURES+NEW_NUM_FEATURES+NEW_CAT_FEATURES], train_split_offer[TARGET]
X_val_offer, y_val_offer = val_split_offer[NUM_FEATURES+CATEGORICAL_OHE_FEATURES+CATEGORICAL_STE_FEATURES+FE+RADUIS_FEATURES+NEW_NUM_FEATURES+NEW_CAT_FEATURES], val_split_offer[TARGET]

In [27]:
X_train_manual, y_train_manual =  train_split_manual[NUM_FEATURES+CATEGORICAL_OHE_FEATURES+CATEGORICAL_STE_FEATURES+FE+RADUIS_FEATURES+NEW_NUM_FEATURES+NEW_CAT_FEATURES], train_split_manual[TARGET]
X_val_manual, y_val_manual = val_split_manual[NUM_FEATURES+CATEGORICAL_OHE_FEATURES+CATEGORICAL_STE_FEATURES+FE+RADUIS_FEATURES+NEW_NUM_FEATURES+NEW_CAT_FEATURES], val_split_manual[TARGET]

In [28]:
model = LGBMRegressor(**MODEL_PARAMS)

In [29]:
model.fit(X_train_offer, y_train_offer)

LGBMRegressor(importance_type='gain', learning_rate=0.01, min_child_samples=5,
              n_estimators=5000, num_leaves=40, random_state=563, reg_alpha=1)

In [30]:
metrics_stat(model.predict(X_val_offer), y_val_offer.values)

{'mape': 1.1812248662681124,
 'mdape': 0.20066081644416806,
 'rmse': 73040.07651460693,
 'r2': 0.7726855965826511,
 'raif_metric': 2.4271719891839805}

In [31]:
metrics_stat(model.predict(X_train_manual), y_train_manual.values)

{'mape': 0.34838046568635095,
 'mdape': 0.2590991414274834,
 'rmse': 73353.86219848657,
 'r2': -0.41611319624099385,
 'raif_metric': 2.2910854048137974}

In [34]:
predictions = model.predict(X_train_manual)
deviation = np.median((y_train_manual.values - predictions)/predictions)
deviation

-0.10207064268943308

In [35]:
preds_manual = model.predict(X_val_manual) * (1 + deviation)
metrics_stat(preds_manual, y_val_manual.values)

{'mape': 0.3785271767980311,
 'mdape': 0.24981135467664162,
 'rmse': 82388.32520812146,
 'r2': -1.4578254104837813,
 'raif_metric': 2.247058793395945}